In [1]:
from module_path_resolution import add_custom_module_path
# So we can import modules including the project folder
add_custom_module_path()
# Know what we're going to use indeed!
import pandas as pd
from pathlib import Path
from pyfriends.core import retrieve_episode_details
from pyfriends.core import SceneCategory
from pyfriends.database_utils import generate_connection
from pyfriends.database_utils import retrieve_engine
from pyfriends.database_utils import execute_query
from pyfriends import tvmaze

# Reading HTML

We should read all the HTML files, parse them to extract everything we can in terms of metadata, and then create a data structure to be used for our DataFrames.

In [2]:
# From 1 to 10 seasons
seasons_identifiers = range(1, 11)
# Main characters
friends = ["CHANDLER", "JOEY", "MONICA", "PHOEBE", "RACHEL", "ROSS"]

In [3]:
seasons_with_episodes = {}

# Will create our dict of seasons with their episodes as generators
for season_identifier in seasons_identifiers:
    episodes = retrieve_episode_details(season_identifier)
    seasons_with_episodes[season_identifier] = episodes
    
seasons_with_episodes

{1: <generator object retrieve_episode_details at 0x7fdd1c1a1430>,
 2: <generator object retrieve_episode_details at 0x7fdd1c1a1580>,
 3: <generator object retrieve_episode_details at 0x7fdd1c1a1660>,
 4: <generator object retrieve_episode_details at 0x7fdd1c1a15f0>,
 5: <generator object retrieve_episode_details at 0x7fdd1c1a1740>,
 6: <generator object retrieve_episode_details at 0x7fdd1c1a17b0>,
 7: <generator object retrieve_episode_details at 0x7fdd1c1a1820>,
 8: <generator object retrieve_episode_details at 0x7fdd1c1a1890>,
 9: <generator object retrieve_episode_details at 0x7fdd1c1a1900>,
 10: <generator object retrieve_episode_details at 0x7fdd1c1a1970>}

In [4]:
# Data that will be used to create our DF!
episode_df_data = []
dialogue_df_data = []
temp_scene_df_data = set()
temp_character_df_data = {}

# Time to fill our DFs!
for season_number, episodes in seasons_with_episodes.items():
    for episode in episodes:
        ## Gathering data for EPISODE DF
        two_part_episode = "/" in episode.number
        total_scenes = len(episode.scenes)
        has_before_opening = any([scene for scene in episode.scenes if scene.category == SceneCategory.BEFORE_OPENING])
        has_after_closing_credits = any([scene for scene in episode.scenes if scene.category == SceneCategory.AFTER_CLOSING_CREDITS])
        # ROW
        entry_episode = [season_number, episode.number, episode.title, two_part_episode, total_scenes, has_before_opening, has_after_closing_credits]
        episode_df_data.append(entry_episode)
        ## Gathering data for DIALOGUE, CHARACTER, and SCENE DF
        transcription_order = 1
        for scene_order, scene in enumerate(episode.scenes, start=1):
            for transcription_order_per_scene, transcription in enumerate(scene.transcriptions, start=1):
                # ROW for CHARACTER DF
                character_name = transcription.character.upper()
                is_main = character_name in friends
                temp_character_df_data[character_name] = is_main
                # ROW for DIALOGUE DF
                entry_dialogue = [season_number, episode.number, character_name, transcription_order, transcription_order_per_scene, transcription.line, scene.description, scene.category.name, scene_order]
                dialogue_df_data.append(entry_dialogue)
                # ROW for SCENE DF
                entry_scene = (season_number, episode.number, scene.description, scene.category.name, scene_order)
                temp_scene_df_data.add(entry_scene)
                # Updating control variable
                transcription_order += 1

In [5]:
# Preparing data
scene_df_data = [[*entry_scene] for entry_scene in temp_scene_df_data]
character_df_data = [[character_name, is_main] for character_name, is_main in temp_character_df_data.items()]

# Creating DataFrames and saving them as parquet files

At this point, we have 4 lists that contain rows:

- episode_df_data
- dialogue_df_data
- character_df_data
- scene_df_data

Let's use them to create our DFs followed by their export into parquet files!

In [6]:
episode_df_columns = [ "SEASON_NUMBER", "NUMBER", "TITLE", "TWO_PART_EPISODE", "TOTAL_SCENES", "HAS_BEFORE_OPENING", "HAS_AFTER_CLOSING_CREDITS" ]
episode_df = pd.DataFrame(episode_df_data, columns = episode_df_columns)

dialogue_df_columns = [ "SEASON_NUMBER", "EPISODE_NUMBER", "CHARACTER_NAME", "TRANSCRIPTION_ORDER", "TRANSCRIPTION_ORDER_PER_SCENE", "TRANSCRIPTION_LINE", "SCENE_DESCRIPTION", "SCENE_CATEGORY", "SCENE_ORDER" ]
dialogue_df = pd.DataFrame(dialogue_df_data, columns = dialogue_df_columns)

character_df_columns = [ "NAME", "IS_MAIN" ]
character_df = pd.DataFrame(character_df_data, columns = character_df_columns)

scene_df_columns = [ "SEASON_NUMBER", "EPISODE_NUMBER", "SCENE_DESCRIPTION", "SCENE_CATEGORY", "SCENE_ORDER" ]
scene_df = pd.DataFrame(scene_df_data, columns = scene_df_columns)

In [7]:
episode_df

,SEASON_NUMBER,NUMBER,TITLE,TWO_PART_EPISODE,TOTAL_SCENES,HAS_BEFORE_OPENING,HAS_AFTER_CLOSING_CREDITS
0,1,24,The One Where Rachel Finds Out,False,11,True,False
1,1,07,The One With the Blackout,False,21,True,True
2,1,19,The One Where the Monkey Gets Away,False,9,True,True
3,1,15,The One With the Stoned Guy,False,14,True,True
4,1,13,The One With the Boobies,False,13,True,True
...,...,...,...,...,...,...,...
222,10,02,The One Where Ross Is Fine,False,12,True,True
223,10,17/18,The Last One,True,25,True,False
224,10,11,The One Where The Stripper Cries,False,14,True,False
225,10,09,The One With The Birth Mother,False,14,True,False


In [8]:
dialogue_df

,SEASON_NUMBER,EPISODE_NUMBER,CHARACTER_NAME,TRANSCRIPTION_ORDER,TRANSCRIPTION_ORDER_PER_SCENE,TRANSCRIPTION_LINE,SCENE_DESCRIPTION,SCENE_CATEGORY,SCENE_ORDER
0,1,24,ROSS,1,1,And here's little Ben nodding off...,"Central Perk, the whole gang is there, Ross is...",BEFORE_OPENING,1
1,1,24,MONICA,2,2,"Awww, look at Aunt Monica's little boy!","Central Perk, the whole gang is there, Ross is...",BEFORE_OPENING,1
2,1,24,PHOEBE,3,3,"Oh, look, he's got Ross's haircut!","Central Perk, the whole gang is there, Ross is...",BEFORE_OPENING,1
3,1,24,RACHEL,4,4,"Oh, let me see! (grabs picture) Oh, God, is he...","Central Perk, the whole gang is there, Ross is...",BEFORE_OPENING,1
4,1,24,ROSS,5,5,(quietly) That would be nice.,"Central Perk, the whole gang is there, Ross is...",BEFORE_OPENING,1
...,...,...,...,...,...,...,...,...,...
61100,10,03,CHANDLER,252,3,"I know, I went to the tanning place and the sa...",Ross's apartment.,AFTER_CLOSING_CREDITS,13
61101,10,03,ROSS,253,4,Really? Did you count Mississipily?,Ross's apartment.,AFTER_CLOSING_CREDITS,13
61102,10,03,ROSS,254,5,"Dude, you're not tanned.",Ross's apartment.,AFTER_CLOSING_CREDITS,13
61103,10,03,CHANDLER,255,6,"No, I just had to get a picture of this.",Ross's apartment.,AFTER_CLOSING_CREDITS,13


In [9]:
character_df

,NAME,IS_MAIN
0,ROSS,True
1,MONICA,True
2,PHOEBE,True
3,RACHEL,True
4,CHANDLER,True
...,...,...
916,PRESENT CHANDLER'S VOICE,False
917,AGENCY GUY,False
918,ADOPTION AGENCY GUY,False
919,ASSISTANT,False


In [10]:
scene_df.query("EPISODE_NUMBER == '01' and SEASON_NUMBER == 1").sort_values(by=['SCENE_ORDER']).head(15)

,SEASON_NUMBER,EPISODE_NUMBER,SCENE_DESCRIPTION,SCENE_CATEGORY,SCENE_ORDER
400,1,01,"Central Perk, Chandler, Joey, Phoebe, and Moni...",MAIN,1
476,1,01,"Monica's Apartment, everyone is there and watc...",MAIN,2
1031,1,01,"The Subway, Phoebe is singing for change.",MAIN,3
1546,1,01,"Ross's Apartment, the guys are there assemblin...",MAIN,4
2874,1,01,"A Restaurant, Monica and Paul are eating.",MAIN,5
1147,1,01,"Monica's Apartment, Rachel is talking on the p...",MAIN,6
780,1,01,Ross's Apartment; Ross is pacing while Joey an...,MAIN,7
345,1,01,"A Restaurant, Monica and Paul are still eating.",MAIN,8
1683,1,01,"Monica's Apartment, Rachel is watching Joanne ...",MAIN,9
3043,1,01,"Ross's Apartment, they're all sitting around a...",MAIN,10


In [11]:
# Saving parquet files
# https://en.wikipedia.org/wiki/Apache_Parquet
folder_where_it_is_running = Path.cwd()
folder_to_save = folder_where_it_is_running.joinpath("integration_layer")

episode_df.to_parquet(f"{folder_to_save}/episode.parquet")
dialogue_df.to_parquet(f"{folder_to_save}/dialogue.parquet")
character_df.to_parquet(f"{folder_to_save}/character.parquet")
scene_df.to_parquet(f"{folder_to_save}/scene.parquet")

# Creating SQL database and enriching it with DataFrames and TVMaze API

With the SQL database, we can configure the data how it really is organized and apply some rules. To illustrate:

- The cardinality between entities: one example is that 1 season has many episodes, and 1 episode belongs to 1 season only.
- Constraints: for instance, 1 dialogue must have a character; thus, it can't be null, the same for its scene; it must be identifiable.

## DDL

Let's build our database! First let's define the [DDL](https://en.wikipedia.org/wiki/Data_definition_language) of it:

In [12]:
entire_ddl = """
-- ---------------------------------------------
-- CLEANING UP

DROP TABLE IF EXISTS character CASCADE;
DROP TABLE IF EXISTS dialogue CASCADE;
DROP TABLE IF EXISTS scene CASCADE;
DROP TABLE IF EXISTS episode CASCADE;
DROP TABLE IF EXISTS season CASCADE;
DROP TABLE IF EXISTS show CASCADE;

-- ---------------------------------------------
-- ALL TABLES

CREATE TABLE IF NOT EXISTS show
(
    id        INT GENERATED ALWAYS AS IDENTITY,
    name      VARCHAR(255) NOT NULL UNIQUE,
    premiered DATE         NOT NULL,
    summary   TEXT         NOT NULL,
    network   VARCHAR(255) NOT NULL,
    country   VARCHAR(255) NOT NULL,
    PRIMARY KEY (id)
);

CREATE TABLE IF NOT EXISTS season
(
    id        INT GENERATED ALWAYS AS IDENTITY,
    number    SMALLINT NOT NULL,
    premiered DATE     NOT NULL,
    end_date  DATE     NOT NULL,
    PRIMARY KEY (id)
);

CREATE TABLE IF NOT EXISTS episode
(
    id               INT GENERATED ALWAYS AS IDENTITY,
    number           VARCHAR(5)   NOT NULL,
    air_date         DATE         NULL,
    title            VARCHAR(255) NULL,
    summary          TEXT         NULL,
    PRIMARY KEY (id)
);

CREATE TABLE IF NOT EXISTS scene
(
    id          INT GENERATED ALWAYS AS IDENTITY,
    number      SMALLINT     NOT NULL,
    description TEXT         NOT NULL,
    category    VARCHAR(255) NOT NULL,
    PRIMARY KEY (id)
);

CREATE TABLE IF NOT EXISTS dialogue
(
    id     INT GENERATED ALWAYS AS IDENTITY,
    number SMALLINT NOT NULL,
    text   TEXT     NOT NULL,
    PRIMARY KEY (id)
);

CREATE TABLE IF NOT EXISTS character
(
    id         INT GENERATED ALWAYS AS IDENTITY,
    short_name VARCHAR(255) NOT NULL UNIQUE,
    PRIMARY KEY (id)
);

-- ---------------------------------------------
-- FOREIGN KEY CONSTRAINTS

ALTER TABLE dialogue
    ADD COLUMN scene_id INTEGER NOT NULL,
    ADD COLUMN character_id INTEGER NULL;

ALTER TABLE dialogue
    ADD CONSTRAINT constraint_fk_scene
        FOREIGN KEY (scene_id)
            REFERENCES scene (id)
            ON DELETE CASCADE,
    ADD CONSTRAINT constraint_fk_character
        FOREIGN KEY (character_id)
            REFERENCES character (id)
            ON DELETE CASCADE;

ALTER TABLE scene
    ADD COLUMN episode_id INTEGER NOT NULL;

ALTER TABLE scene
    ADD CONSTRAINT constraint_fk
        FOREIGN KEY (episode_id)
            REFERENCES episode (id)
            ON DELETE CASCADE;

ALTER TABLE episode
    ADD COLUMN season_id INTEGER NOT NULL;

ALTER TABLE episode
    ADD CONSTRAINT constraint_fk
        FOREIGN KEY (season_id)
            REFERENCES season (id)
            ON DELETE CASCADE;

ALTER TABLE season
    ADD COLUMN show_id INTEGER NOT NULL;

ALTER TABLE season
    ADD CONSTRAINT constraint_fk
        FOREIGN KEY (show_id)
            REFERENCES show (id)
            ON DELETE CASCADE;

-- ---------------------------------------------
-- OTHER CONSTRAINTS APART FROM FOREIGN KEY

ALTER TABLE season ADD CONSTRAINT unique_season_per_show UNIQUE (number, show_id);
ALTER TABLE episode ADD CONSTRAINT unique_episode_per_season UNIQUE (number, season_id);
ALTER TABLE scene ADD CONSTRAINT unique_scene_per_episode UNIQUE (number, episode_id);
ALTER TABLE dialogue ADD CONSTRAINT unique_dialogue_per_scene_character UNIQUE(scene_id, number, character_id);
"""

with generate_connection() as connection:
    cursor = connection.cursor()
    try:
        cursor.execute(entire_ddl)
    finally:
        connection.commit()

## DML

Now we can use the dataframes to fill all the tables. Still, we should take care because you might insert duplicated content and then raise an exception because of the constraints we created.

### Show table

In [13]:
# This is required so to_sql can work properly with postgresql
engine = retrieve_engine()

# Insert details about the Friends show from TVMaze database
# https://www.tvmaze.com/shows/431/friends
friends_id = 431

show = tvmaze.show_details(friends_id)

data = {
    "name": [show.name],
    "premiered": [show.premiered],
    "summary": [show.summary],
    "network": [show.network.name],
    "country": [show.network.country],
}

show_df = pd.DataFrame(data)

show_df.to_sql("show", engine, if_exists="append", index=False)
engine.dispose()

### Season table

In [14]:
seasons = tvmaze.all_seasons(friends_id)

# Just showing how to get the ID dynamically
show_id = execute_query("SELECT id FROM show WHERE lower(name) = 'friends'")[0][0]

data = {
    "number": [season.number for season in seasons],
    "premiered": [season.premiered_date for season in seasons],
    "end_date": [season.end_date for season in seasons],
    "show_id": [show_id for _ in seasons],
}

season_df = pd.DataFrame(data)

season_df.to_sql("season", engine, if_exists="append", index=False)
engine.dispose()

### Episode table

The cell below delays a bit to finish because of some REST API calls 😅. If you don't want Friends database to be enriched, just set `include_enriched_data_from_tvmaze` as `False`.

In [15]:
include_enriched_data_from_tvmaze = False
# Let's create a new custom DF to fill our table! Again, we'll use TVMaze API to enrich it
# First let's build all the rows
custom_episode_df_data = []
    
for _, row in episode_df.iterrows():
    # Season ID from the database
    season_number = row["SEASON_NUMBER"]
    raw_query_to_retrieve_season_id = f"SELECT id FROM season WHERE number = {season_number}"
    season_id = execute_query(raw_query_to_retrieve_season_id)[0][0]
    # More details about the episode thanks to TVMaze
    episode_number = row["NUMBER"]
    episode_details = None
    if include_enriched_data_from_tvmaze:
        if "/" in episode_number:
            episode_number_part_1, episode_number_part_2 = episode_number.split("/")
            episode_part_1_details = tvmaze.episode_details(friends_id, season_number, episode_number_part_1)
            episode_part_2_details = tvmaze.episode_details(friends_id, season_number, episode_number_part_2)
            # Merge into one object
            # For this particular case, we only care about summary attribute
            final_summary = f"{episode_part_1_details.summary} {episode_part_2_details.summary}"        
            episode_details = tvmaze.Episode(episode_part_1_details.title, episode_part_1_details.air_date, episode_part_1_details.runtime, final_summary, episode_part_1_details.type)
        else:
            episode_details = tvmaze.episode_details(friends_id, season_number, episode_number)
    # The entry for the temporary DF
    episode_title = row["TITLE"]
    entry = None
    if include_enriched_data_from_tvmaze:
        entry = [episode_number, episode_details.air_date, episode_title, episode_details.summary, season_id]
    else:
        entry = [episode_number, None, episode_title, None, season_id]
    custom_episode_df_data.append(entry)

In [16]:
custom_episode_df_column = ["number", "air_date", "title", "summary", "season_id" ]
custom_episode_df = pd.DataFrame(custom_episode_df_data, columns = custom_episode_df_column)

custom_episode_df.to_sql("episode", engine, if_exists="append", index=False)
engine.dispose()

### Scene, dialogue, and character tables

In [17]:
def retrieve_episode_id_by_season_and_episode_number(season_number, episode_number, connection):
    raw_query = f"SELECT ep.id FROM episode ep INNER JOIN season se on se.id = ep.season_id WHERE se.number = {season_number} AND ep.number = '{episode_number}'"
    return int(execute_query(raw_query, connection=connection)[0][0])

def retrieve_scene_id_by(episode_id, scene_number, connection):
    raw_query = f"SELECT sc.id FROM scene sc INNER JOIN episode ep on ep.id = sc.episode_id WHERE ep.id = {episode_id} AND sc.number = {scene_number}"
    return int(execute_query(raw_query, connection=connection)[0][0])

# Control variable
repositories_of_characters = {}

def persist_if_applicable_and_retrieve_character_id(character_name, connection):
    if repositories_of_characters.get(character_name):
        return repositories_of_characters[character_name]
    else:
        temp_character_df = pd.DataFrame([[character_name]], columns = ["short_name"])
        temp_character_df.to_sql("character", connection, if_exists="append", index=False)
        raw_query = r"SELECT id FROM character WHERE short_name = :short_name"
        data = {"short_name": character_name}
        repositories_of_characters[character_name] = int(execute_query(raw_query, connection=connection, data=data)[0][0])
        return repositories_of_characters[character_name]

# All the rows to dialogue table. Scene table will be filled on the fly!
custom_dialogue_df_data = []

def fill_scene_table_and_create_data_to_be_inserted_into_dialogue_table(connection):
    for season_identifier in seasons_identifiers:
        filtered_df_by_season = dialogue_df.query(f"SEASON_NUMBER == {season_identifier}")
        ordered_df_by_episode_and_scene = filtered_df_by_season.sort_values(by=['EPISODE_NUMBER', 'SCENE_ORDER'])
        # Control variables
        current_episode_number = None
        current_scene_order = None
        # Database IDs variables
        episode_id = None
        scene_id = None
        for _, row in ordered_df_by_episode_and_scene.iterrows():
            episode_number = row["EPISODE_NUMBER"]
            scene_order = row["SCENE_ORDER"]
            dialogue_order = row["TRANSCRIPTION_ORDER"]
            # So we can restart the dialogue order control variable
            if episode_number != current_episode_number:                
                current_episode_number = episode_number
                episode_id = retrieve_episode_id_by_season_and_episode_number(season_identifier, current_episode_number, connection)
            # Create or not a new scene given it has been changed
            scene_description = row["SCENE_DESCRIPTION"]
            scene_category = row["SCENE_CATEGORY"]
            if scene_order != current_scene_order:                
                current_scene_order = scene_order
                temp_scene_df_entry = [scene_order, scene_description, scene_category, episode_id]
                temp_scene_df = pd.DataFrame([temp_scene_df_entry], columns = ["number", "description", "category", "episode_id"])
                temp_scene_df.to_sql("scene", connection, if_exists="append", index=False)
                scene_id = retrieve_scene_id_by(episode_id, scene_order, connection)
            # Finally the entry for the dialogue
            text = row["TRANSCRIPTION_LINE"]
            character_name = row["CHARACTER_NAME"]
            character_id = persist_if_applicable_and_retrieve_character_id(character_name, connection)
            custom_dialogue_df_entry = [dialogue_order, text, scene_id, character_id]
            custom_dialogue_df_data.append(custom_dialogue_df_entry)

with engine.connect() as connection:
    fill_scene_table_and_create_data_to_be_inserted_into_dialogue_table(connection)

In [18]:
custom_dialogue_df_column = ["number", "text", "scene_id", "character_id"]
custom_dialogue_df = pd.DataFrame(custom_dialogue_df_data, columns = custom_dialogue_df_column)

custom_dialogue_df.to_sql("dialogue", engine, if_exists="append", index=False)
engine.dispose()